In [1]:
import random   

# initialize some useful global variables
in_play = False

# define globals for cards
SUITS = ('C', 'S', 'H', 'D')
RANKS = ('A', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K')
VALUES = {'A':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':10, 'Q':10, 'K':10}

In [2]:
# define card class
class Card:
    def __init__(self, suit, rank):
        if (suit in SUITS) and (rank in RANKS):
            self.suit = suit
            self.rank = rank
        else:
            self.suit = None
            self.rank = None
            print "Invalid card: ", suit, rank

    def __str__(self):
        return self.suit + self.rank

    def get_suit(self):
        return self.suit

    def get_rank(self):
        return self.rank

In [3]:
# define hand class
class Hand:
    def __init__(self):
        self.hand_list = []	# create Hand object

    def __str__(self):
        ans = ""
        for i in range(len(self.hand_list)):
            ans = ans +' '+ str(self.hand_list[i])
        return 'Hand contains'+ans	# return a string representation of a hand

    def add_card(self, card):
        self.hand_list.append(card)	# add a card object to a hand

    def get_value(self):
        global got_aces #this variable is used to identify if a hand is hard (value = 0) or soft (value = 1)
        value = 0
        got_aces = 0
        for i in range(len(self.hand_list)):
                v = str(self.hand_list[i])
                v = v[1:]
                if v =='A':
                    got_aces = 1
                value += VALUES.get(v)
        if got_aces == 1 and value + 10 > 21:
            got_aces = 0
        elif got_aces == 1 and value + 10 <=21:
            value = value + 10
        return value	# compute the value of the hand
    
    def get_aces(self):
        #computes the number of aces in hand
        aces = 0
        for i in range(len(self.hand_list)):
            v = str(self.hand_list[i])[1:]
            if v == 'A':
                aces += 1
        return aces
    
    def __len__(self):
        return len(self.hand_list)
    
    def __getitem__(self, key):
        return str(self.hand_list[key])[1:]

In [4]:
# define deck class 
class Deck:
    def __init__(self):
        self.deck_list=[]
        for i in range(len(SUITS)):
            for j in range(len(RANKS)):
                c = Card(SUITS[i], RANKS[j])
                self.deck_list.append(c)
                # create a Deck object

    def shuffle(self):
        # shuffle the deck 
        random.shuffle(self.deck_list)    

    def deal_card(self):
        deal = self.deck_list[-1]
        self.deck_list.pop(-1)
        return deal
        
    
    def __str__(self):
        ans = ""
        for i in range(len(self.deck_list)):
            ans = ans +' '+ str(self.deck_list[i])
        return 'Deck contains'+ans
    

In [5]:
#define event handlers
def deal():
    """this function is used for dealing cards at the beginning of the game"""
    global in_play, deck, player_hand, dealer_hand
    deck = Deck()
    deck.shuffle()
    player_hand = Hand()
    dealer_hand = Hand()
    for i in range(2):
        player_hand.add_card(deck.deal_card())
        dealer_hand.add_card(deck.deal_card())   
    in_play = True
    #print(player_hand.__str__())
    #print(dealer_hand.__str__())

    
    
def hit():
    """this function is used for adding a new card to the hand"""
    global in_play, win, lose, tie
    if in_play == True:
        if player_hand.get_value()<=21:
            player_hand.add_card(deck.deal_card())
            if player_hand.get_value()>21:
                lose = lose + 1
                in_play = False               
    print(player_hand.__str__())
    
                
       
def stand():
        """this function is called after the player stands"""
        global in_play, win, lose, tie, initial_hand_value
  
        if initial_hand_value>21:
            in_play = False
            lose = lose + 1
        else:
            while dealer_hand.get_value()<17:
                dealer_hand.add_card(deck.deal_card())
            if dealer_hand.get_value()>21:
                win = win + 1
                in_play = False
            elif dealer_hand.get_value()>initial_hand_value:
                lose = lose + 1
                in_play = False
            elif dealer_hand.get_value()==initial_hand_value:
                tie = tie + 1
                in_play = False
            elif initial_hand_value>dealer_hand.get_value():
                win = win + 1
                in_play = False

In [6]:
def hard_reward(initial_hand_value, action, hand_value_after_action, aces_in_hand, dealer_card):
    """This function is used for computing the reward for hard hands"""
    random_reward = random.uniform(0,1) * 2 - 1
    
    if action == 'Hit':
        if hand_value_after_action < 22:
            reward = random_reward + 2
        else:
            reward = random_reward - 2
    elif action == 'Stand':
        if hand_value_after_action > 21:
            reward = random_reward + 2
        else:
            reward = random_reward - 2
                                
    return reward

In [7]:
def reward(action):
    """This function is used for computing the reward when taking an action in a state"""
    global initial_hand_value
    initial_hand_value = player_hand.get_value()
    chosen_action = action
    aces_in_hand = got_aces
    if aces_in_hand == 1:
        rwrd = soft_reward(action)
    else:
        player_hand.add_card(deck.deal_card())
        hand_value_after_action = player_hand.get_value()
        dealer_card = dealer_hand[-1]
    
        rwrd = hard_reward(initial_hand_value, chosen_action, hand_value_after_action, aces_in_hand, dealer_card)
    
    return rwrd

In [8]:
def soft_reward(action):
    """The soft_reward function is called to compute the reward for player hands that are soft"""
    global got_aces
    chosen_action = action
    random_reward = random.uniform(0,1) * 2 - 1
    initial_soft_hand_value = player_hand.get_value()
    player_hand.add_card(deck.deal_card())
    hand_value_after_hit = player_hand.get_value()
    if got_aces == 1:
        if chosen_action == 'Hit':
            reward = random_reward + 2
        elif chosen_action == 'Stand':
            reward = random_reward - 2
    else:
        while player_hand.get_value() <= initial_soft_hand_value:
            player_hand.add_card(deck.deal_card())
        if player_hand.get_value() > initial_soft_hand_value and player_hand.get_value() < 22:
            if chosen_action == 'Hit':
                reward = random_reward + 2
            elif chosen_action == 'Stand':
                reward = random_reward - 2
        else:
            if chosen_action == 'Hit':
                reward = random_reward - 2
            elif chosen_action == 'Stand':
                reward = random_reward + 2
    return reward

In [9]:
def build_state():
    """This function is called to assemble the state as a tuple of the following 4 parameters:
    1. sum of the cards in hand (player_hand_value)
    2. if the hand is hard (got_aces = 0) or soft (got_aces = 1)
    3. number of cards in hand (len(player_hand))
    4. the dealer card that is visible to the player (dealer_hand[-1])"""
    global got_aces
    player_hand_value = player_hand.get_value()
    state = (player_hand_value, got_aces, len(player_hand), dealer_hand[-1])
    return state

In [10]:
def createQ(state):
    '''if machine is learning and encounters a state that wasn't encountered before, create an entry in the Q-table 
    with default values for each action'''
    if state not in Q:
        Q[state] = {'Hit' : 0.0, 'Stand' : 0.0}

In [11]:
def choose_action(state):
    '''This function is called when the machine has to choose an action in a state'''
    action_list = ['Hit', 'Stand']
            
    if learning:
        action = random.choice(action_list)                
    else:
        createQ(state)
        action = get_action(Q[state], get_maxQ(state))
        
    return action

In [12]:
def get_action(dictionary, value):
        """ The function is called when searching for the action 
        with the highest reward in a state """
        action_list = []
        for k, v in dictionary.iteritems():
            if v == value:
                action_list.append(k)
        return random.choice(action_list)

In [13]:
def get_maxQ(state):
        """ The get_maxQ function is called when the machine is asked to find the
            maximum Q-value of all actions based on the 'state' it is in. """
        maxQ = max(Q[state].values())

        return maxQ 

In [14]:
def learn(state, action, reward):
        """ The learn function is called after the machine completes an action and
            receives an award. This function does not consider future rewards 
            when conducting learning. """
        Q[state][action] += reward
        return

In [15]:
def run():
    """ The run function is used for playing a game of Blackjack"""
    global lose, in_play
    deal()
    in_play = True
    while in_play:
    
        state = build_state()
        createQ(state)
        action = choose_action(state)
        Reward = reward(action)
        learn(state, action, Reward)
        if action == 'Stand':
            stand()
        elif action == 'Hit':
            if player_hand.get_value()>21:
                lose = lose + 1
                in_play = False

In [51]:
#Creating a dictionary that will be used as Q-table
Q = dict()

In [55]:
#The learning flag is set to True for training the model and False for testing the model
learning = False

In [80]:
#Defining the variables that will be used for evaluating the performance of the model
tie = 0 #counts the number of games that end in tie
win = 0 #counts the number of games that end with player winning
lose = 0 #counts the number of games that end with player losing

In [81]:
#Play the number of games equal to the range of i
for i in range (100000):
    run()
print("Done")

Done


In [82]:
#Compute the win percentage of the model
print('Win percentage of the model is: ' + str(float(win) / (win + lose + tie) * 100)) + '%'

Win percentage of the model is: 42.514%


In [190]:
#Verify the entries in the Q-table for specific parameters of the states (e.g sum of cards in hand -- k[0], hard hands or soft hands -- k[1]==0 or k[1]!=0)
for k, v in Q.items():
    if k[0] == 21 and k[1] == 1:
        print(k, v)